# Test query

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

from qdrant_client import QdrantClient
from langchain_openai import AzureOpenAIEmbeddings



def query(query_text: str, limit: int = 5) -> list[str]:
    """
    Query the Qdrant vector database with a text query and return matching results.
    
    Args:
        query_text (str): The text query to search for
        limit (int): Maximum number of results to return (default: 5)
    
    Returns:
        list[str]: List of matching text results
    """
    try:
        # Generate embedding for the query text
        query_embedding = embedding_model.embed_query(query_text)
        
        # Search in Qdrant
        search_results = qdrant_client.search(
            collection_name=COLLECTION_NAME,
            query_vector=query_embedding,
            limit=limit
        )
        
        # Extract text content from results
        results = []
        for result in search_results:
            # Assuming the text content is stored in payload under 'text' key
            # Adjust the key name based on your actual data structure
            if 'text' in result.payload:
                results.append(result.payload['text'])
            elif 'content' in result.payload:
                results.append(result.payload['content'])
            else:
                # If no text field found, convert payload to string
                results.append(str(result.payload))
        
        return results
        
    except Exception as e:
        print(f"Error during query: {e}")
        return []

In [25]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import AzureChatOpenAI
import time
import os

from dotenv import load_dotenv

load_dotenv()

AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL_NAME=os.getenv("AZURE_OPENAI_MODEL_NAME")
AZURE_OPENAI_MODEL_API_VERSION=os.getenv("AZURE_OPENAI_MODEL_API_VERSION")

model = AzureChatOpenAI(
    model=AZURE_OPENAI_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_MODEL_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_tokens=5000
)

client = MultiServerMCPClient(
    {
        "RAGService": {
            "url": "http://localhost:8002/sse",
            "transport": "sse",
        },
        "DocumentService": {
            "url": "http://localhost:8001/sse",
            "transport": "sse",
        }
    }
)

tools = await client.get_tools()
prompt = "You are a RAG agent, please query the RAGService for the information if the user asks"
agent = create_react_agent(model, tools, prompt=prompt)


start_time = time.time()
answer = await astream_graph(
    agent, {"messages": "What's MCP? Answer with out query"}
)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

AttributeError: '_AsyncGeneratorContextManager' object has no attribute 'get_tools'

In [21]:
print(f"Time taken: {end_time - start_time} seconds")

Time taken: 12.563843965530396 seconds


In [9]:
import os
import asyncio
from dotenv import load_dotenv

from langchain_openai import AzureChatOpenAI
from langchain_mcp_adapters.client import MCPClient
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

load_dotenv()

# Azure OpenAI Configuration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL_NAME = os.getenv("AZURE_OPENAI_MODEL_NAME")
AZURE_OPENAI_MODEL_API_VERSION = os.getenv("AZURE_OPENAI_MODEL_API_VERSION")

model = AzureChatOpenAI(
    model=AZURE_OPENAI_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_MODEL_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_tokens=5000
)

async def setup_agents():
    """Setup individual MCP clients and agents"""
    
    # RAG Service MCP Client
    rag_client = MCPClient(
        url="http://localhost:8002/sse",
        transport="sse"
    )
    
    # Document Service MCP Client  
    document_client = MCPClient(
        url="http://localhost:8001/sse",
        transport="sse"
    )
    
    # Get tools from each service
    rag_tools = await rag_client.get_tools()
    document_tools = await document_client.get_tools()
    
    # Create RAG Agent
    rag_agent = create_react_agent(
        model=model,
        tools=rag_tools,
        name="rag_expert",
        prompt=(
            "You are a RAG (Retrieval-Augmented Generation) expert. "
            "Use vector search to find relevant information from the knowledge base. "
            "Provide accurate, contextual answers based on retrieved documents. "
            "Always cite your sources when possible."
        )
    )
    
    # Create Document Agent
    document_agent = create_react_agent(
        model=model,
        tools=document_tools,
        name="document_expert", 
        prompt=(
            "You are a document processing expert. "
            "Handle document operations like reading, parsing, and extracting information. "
            "Process various file formats and provide structured summaries. "
            "Focus on accuracy and completeness in document analysis."
        )
    )
    
    return rag_agent, document_agent

async def create_supervisor_workflow():
    """Create the supervisor workflow with specialized agents"""
    
    # Setup agents
    rag_agent, document_agent = await setup_agents()
    
    # Create supervisor workflow
    workflow = create_supervisor(
        agents=[rag_agent, document_agent],
        model=model,
        prompt=(
            "You are a team supervisor managing specialized AI agents. "
            "Route tasks based on their nature:\n"
            "- For knowledge retrieval, semantic search, or answering questions from existing data: use rag_expert\n"
            "- For document processing, file analysis, or content extraction: use document_expert\n"
            "Always choose the most appropriate agent for the task."
        )
    )
    
    return workflow

async def main():
    """Main execution function"""
    
    # Create supervisor workflow
    workflow = await create_supervisor_workflow()
    
    # Compile the workflow
    app = workflow.compile()
    
    # Example usage
    result = await app.ainvoke({
        "messages": [
            {
                "role": "user", 
                "content": "What's MCP? Please query the RAGService for information."
            }
        ]
    })
    
    print("=== Supervisor Response ===")
    for message in result["messages"]:
        print(f"{message['role']}: {message['content']}")
        print("-" * 50)

# Alternative synchronous wrapper for easier usage
def run_supervisor_query(query: str):
    """Synchronous wrapper for running supervisor queries"""
    
    async def _run():
        workflow = await create_supervisor_workflow()
        app = workflow.compile()
        
        result = await app.ainvoke({
            "messages": [{"role": "user", "content": query}]
        })
        
        return result
    
    return asyncio.run(_run())

# Example usage functions
async def example_rag_query():
    """Example RAG query"""
    workflow = await create_supervisor_workflow()
    app = workflow.compile()
    
    result = await app.ainvoke({
        "messages": [
            {
                "role": "user",
                "content": "Search for information about machine learning algorithms in the knowledge base."
            }
        ]
    })
    
    return result

async def example_document_query():
    """Example document processing query"""
    workflow = await create_supervisor_workflow()
    app = workflow.compile()
    
    result = await app.ainvoke({
        "messages": [
            {
                "role": "user", 
                "content": "Please analyze and summarize the contents of the uploaded PDF document."
            }
        ]
    })
    
    return result

if __name__ == "__main__":
    # Run the main example
    asyncio.run(main())
    
    # Or use the synchronous wrapper
    # result = run_supervisor_query("What's MCP? Please search the knowledge base.")
    # print(result)

Using existing file: mcp.md
Collection 'mcp' does not exist. Creating it.
Extracting text from mcp.md...
Text extracted (first 200 chars): # Meeting Minutes

## June 19, 2025

### Attendees
- Alice
- Bob
- Charlie

### Discussion Points
1.  **Project Alpha**: Reviewed progress. On track for phase 1 completion.
2.  **Budget Review**: Disc...
Processing and adding chunks to Qdrant using 'markdown_header' method...
Successfully upserted 3 chunks to Qdrant for document ID: mcp.md
Document processing complete.


---

In [16]:
import os
import asyncio
from dotenv import load_dotenv

from langchain_openai import AzureChatOpenAI
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

load_dotenv()

# Azure OpenAI Configuration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL_NAME = os.getenv("AZURE_OPENAI_MODEL_NAME")
AZURE_OPENAI_MODEL_API_VERSION = os.getenv("AZURE_OPENAI_MODEL_API_VERSION")

model = AzureChatOpenAI(
    model=AZURE_OPENAI_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_MODEL_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_tokens=5000
)

async def setup_rag_agent():
    """Setup RAG agent with single MCP client"""
    
    # Connect to RAG Service using streamablehttp_client
    async with streamablehttp_client("http://localhost:8002/mcp/") as (read, write, _):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()
            
            # Get tools from RAG service
            rag_tools = await load_mcp_tools(session)
            
            # Create RAG Agent
            rag_agent = create_react_agent(
                model=model,
                tools=rag_tools,
                name="rag_expert",
                prompt=(
                    "You are a RAG (Retrieval-Augmented Generation) expert. "
                    "Use vector search to find relevant information from the knowledge base. "
                    "Provide accurate, contextual answers based on retrieved documents. "
                    "Always cite your sources when possible."
                )
            )
            
            return rag_agent, session  # Return session to keep connection alive

async def create_supervisor_workflow():
    """Create the supervisor workflow with RAG agent only"""
    
    # Setup RAG agent
    rag_agent, session = await setup_rag_agent()
    
    # Create supervisor workflow
    workflow = create_supervisor(
        agents=[rag_agent],
        model=model,
        prompt=(
            "You are a team supervisor managing a RAG expert. "
            "Route knowledge retrieval, semantic search, or question answering tasks to rag_expert. "
            "Always use the rag_expert for information retrieval from the knowledge base."
        )
    )
    
    return workflow, session

# Alternative approach using MultiServerMCPClient for single server
async def setup_rag_agent_alternative():
    """Alternative setup using MultiServerMCPClient with single server"""
    from langchain_mcp_adapters.client import MultiServerMCPClient
    
    # Single server configuration
    client = MultiServerMCPClient({
        "rag_service": {
            "url": "http://localhost:8002/sse",
            "transport": "sse",
        }
    })
    
    # Get tools
    rag_tools = await client.get_tools()
    
    # Create RAG Agent
    rag_agent = create_react_agent(
        model=model,
        tools=rag_tools,
        name="rag_expert",
        prompt=(
            "You are a RAG expert. "
            "Search the knowledge base to answer questions. "
            "Provide accurate answers with source citations."
        )
    )
    
    return rag_agent

async def create_supervisor_workflow_alternative():
    """Alternative supervisor workflow"""
    
    # Setup RAG agent
    rag_agent = await setup_rag_agent_alternative()
    
    # Create supervisor workflow
    workflow = create_supervisor(
        agents=[rag_agent],
        model=model,
        prompt=(
            "You manage a RAG expert. "
            "For any knowledge questions or information retrieval: use rag_expert."
        )
    )
    
    return workflow

async def main():
    """Main execution function - using alternative approach"""
    
    # Create supervisor workflow (using simpler MultiServerMCPClient approach)
    workflow = await create_supervisor_workflow_alternative()
    
    # Compile the workflow
    app = workflow.compile()
    
    # Example usage
    result = await app.ainvoke({
        "messages": [
            {
                "role": "user", 
                "content": "What's MCP? Please query the RAGService for information."
            }
        ]
    })
    
    print("=== Supervisor Response ===")
    for message in result["messages"]:
        print(f"{message['role']}: {message['content']}")
        print("-" * 50)

# Synchronous wrapper
def run_supervisor_query(query: str):
    """Synchronous wrapper for running supervisor queries"""
    
    async def _run():
        workflow = await create_supervisor_workflow_alternative()
        app = workflow.compile()
        
        result = await app.ainvoke({
            "messages": [{"role": "user", "content": query}]
        })
        
        return result
    
    return asyncio.run(_run())

# Test functions
async def test_rag_query():
    """Test RAG query"""
    workflow = await create_supervisor_workflow_alternative()
    app = workflow.compile()
    
    result = await app.ainvoke({
        "messages": [
            {
                "role": "user",
                "content": "Please tell me what is Model Context Protocol?"
            }
        ]
    })
    
    print("=== RAG Test Response ===")
    for message in result["messages"]:
        print(f"{message.role}: {message.content}")
        print("-" * 50)
    
    return result



if __name__ == "__main__":
    # This only runs when executed as a script, not in Jupyter
    await main()

=== Supervisor Response ===


TypeError: 'HumanMessage' object is not subscriptable

In [ ]:
import os
import asyncio
from datetime import datetime
from dotenv import load_dotenv

from langchain_openai import AzureChatOpenAI
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

load_dotenv()

# Azure OpenAI Configuration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL_NAME = os.getenv("AZURE_OPENAI_MODEL_NAME")
AZURE_OPENAI_MODEL_API_VERSION = os.getenv("AZURE_OPENAI_MODEL_API_VERSION")

model = AzureChatOpenAI(
    model=AZURE_OPENAI_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_MODEL_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_tokens=5000
)

def log_time(message, start=None):
    now = datetime.now()
    if start:
        duration = now - start
        print(f"[{now}] {message} - Duration: {duration}")
    else:
        print(f"[{now}] {message}")
    return now

async def setup_rag_agent_alternative():
    from langchain_mcp_adapters.client import MultiServerMCPClient
    
    log_time("Initializing MultiServerMCPClient")
    start = datetime.now()

    client = MultiServerMCPClient({
        "rag_service": {
            "url": "http://localhost:8002/sse",
            "transport": "sse",
        }
    })

    log_time("Fetching RAG tools from MCP", start)
    start = datetime.now()
    rag_tools = await client.get_tools()
    log_time("Fetched RAG tools", start)

    log_time("Creating RAG Agent")
    start = datetime.now()
    rag_agent = create_react_agent(
        model=model,
        tools=rag_tools,
        name="rag_expert",
        prompt=(
            "You are a RAG expert. "
            "Search the knowledge base to answer questions. "
            "Provide accurate answers with source citations."
        )
    )
    log_time("Created RAG Agent", start)
    return rag_agent

async def create_supervisor_workflow_alternative():
    log_time("Setting up RAG Agent (Alternative)")
    start = datetime.now()
    rag_agent = await setup_rag_agent_alternative()
    log_time("RAG Agent setup complete", start)

    log_time("Creating Supervisor Workflow")
    start = datetime.now()
    workflow = create_supervisor(
        agents=[rag_agent],
        model=model,
        prompt=(
            "You manage a RAG expert. "
            "For any knowledge questions or information retrieval: use rag_expert."
        )
    )
    log_time("Created Supervisor Workflow", start)
    return workflow

async def main():
    log_time("Main execution started")
    start_total = datetime.now()

    workflow = await create_supervisor_workflow_alternative()
    
    log_time("Compiling workflow")
    start = datetime.now()
    app = workflow.compile()
    log_time("Workflow compiled", start)

    log_time("Sending user query to app")
    start = datetime.now()
    result = await app.ainvoke({
        "messages": [
            {
                "role": "user", 
                "content": "What's MCP? Please query the RAGService for information."
            }
        ]
    })
    log_time("Query completed", start)

    print("\n=== Supervisor Response ===")
    for message in result["messages"]:
        print(f"{message['role']}: {message['content']}")
        print("-" * 50)

    log_time("Main execution finished", start_total)

def run_supervisor_query(query: str):
    async def _run():
        workflow = await create_supervisor_workflow_alternative()
        app = workflow.compile()
        
        start = datetime.now()
        result = await app.ainvoke({
            "messages": [{"role": "user", "content": query}]
        })
        log_time("Synchronous query completed", start)

        return result

    return asyncio.run(_run())

if __name__ == "__main__":
    await main()


In [1]:
import pandas as pd

In [3]:
path = "web_sk.csv"

df = pd.read_csv(path)

In [5]:
df_sk = df[df['Web'].str.contains('.sk')]


In [7]:
# only keep website that contain .sk in the url Web

df_sk.to_csv("web_sk_categorized.csv", index=False)